In [12]:
# Visualize the points from the dataset

import cv2
import numpy as np
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
import random
import pandas as pd
import torch

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [13]:
data = pd.read_csv("data.csv")

In [14]:
def showLandmarksFromDataset(index):
  blank_frame = np.zeros((256, 256, 3), dtype=np.uint8)

  # List of landmark names in the order MediaPipe expects
  landmark_names = [
      'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer',
      'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow',
      'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index',
      'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle',
      'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'
  ]

  landmarks = []
  row = data.iloc[index]
  for name in landmark_names:
      x = row[f'{name}_x']
      y = row[f'{name}_y']
      z = row[f'{name}_z']

      x = (x + 1.0) / 2.0
      y = (y + 1.0) / 2.0
      z = z

      x = np.clip(x, 0.0, 1.0)
      y = np.clip(y, 0.0, 1.0)
      z = np.clip(z, -1.0, 1.0)

      landmarks.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z))

  landmark_list = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)

  # Mapping from label to full posture name
  label_to_name = {
      "TUP": "Upright trunk position",
      "TLB": "Trunk leaning backward",
      "TLF": "Trunk leaning forward",
      "TLR": "Trunk leaning right",
      "TLL": "Trunk leaning left"
  }

  label = row['upperbody_label']
  full_name = label_to_name.get(label, label)

  mp_drawing.draw_landmarks(
    image=blank_frame,
    landmark_list=landmark_list,
    connections=mp_pose.POSE_CONNECTIONS,  # Use the appropriate connections for your landmarks
    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
    connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,0,0), thickness=2)
  )

  cv2.imshow('Landmarks', blank_frame)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [15]:
# for i in range(0, 4000):
#   showLandmarksFromDataset(i)

In [16]:
import cv2
import numpy as np
import mediapipe as mp

In [17]:
def getLabel(fileName):
  mp_pose = mp.solutions.pose
  pose = mp_pose.Pose(min_detection_confidence=0.7,
                        min_tracking_confidence=0.5)
  mp_drawing = mp.solutions.drawing_utils

  img = cv2.imread(fileName)
  rgb_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  results = pose.process(rgb_frame)

  if results.pose_landmarks:
    mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

  cv2.imshow('MediaPipe Pose', img)
  cv2.waitKey(0)  # Waits indefinitely until a key is pressed
  pose.close()
  cv2.destroyAllWindows()

  X_test = [[]]
  for i, landmark in enumerate(results.pose_landmarks.landmark):
    X_test[0].append(landmark.x)
    X_test[0].append(landmark.y)
    X_test[0].append(landmark.z)
    X_test[0].append(landmark.visibility)
  X_test, len(X_test[0])

  model = torch.load("model_0.pth", weights_only=False)
  model.eval()
  X_test = torch.tensor(X_test, dtype=torch.float).to(device="cuda")
  with torch.inference_mode():
    y_logits = model(X_test)
  y_logits

  y_preds_probs = torch.softmax(y_logits, dim=1)
  return torch.argmax(y_preds_probs)

In [25]:
getLabel("../data/test7.jpg")

tensor(0, device='cuda:0')

In [ ]:
for i in range(1, 21):
  if getLabel(f"../data/good/goodposture{i}.jpg") != 4:
    print("woah")

In [ ]:
X_test = [[]]
for landmark in upper_landmarks:
  X_test[0].append(landmark.x)
  X_test[0].append(landmark.y)
  X_test[0].append(landmark.z)
X_test, len(X_test[0])

NameError: name 'upper_landmarks' is not defined

In [ ]:
model = torch.load("model_0.pth", weights_only=False)
model.eval()
X_test = torch.tensor(X_test, dtype=torch.float).to(device="cuda")
with torch.inference_mode():
  y_logits = model(X_test)
y_logits

tensor([[ -1.9885,  -2.7045,  -4.0109, -19.2214,  30.3151]], device='cuda:0')

In [ ]:
y_preds_probs = torch.softmax(y_logits, dim=1)
y_preds_probs

tensor([[9.3485e-15, 4.5685e-15, 1.2371e-15, 3.0662e-22, 1.0000e+00]],
       device='cuda:0')

In [ ]:
torch.argmax(y_preds_probs)

tensor(4, device='cuda:0')